In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os, sys
import sys
from torchsummary import summary
# sys.path.append('/home/po/TM5/Pointnet_Pointnet2_pytorch')
sys.path.append('/home/po/TM5/s4g-release/inference/grasp_proposal/network_models')
sys.path.append('/home/po/TM5/graspnetAPI/graspnetAPI')
sys.path.append('/home/po/TM5')
from nn_utils.mlp import SharedMLP
from pointnet2_utils.modules import PointNetSAModule, PointnetFPModule, PointNetSAAvgModule
from nn_utils.functional import smooth_cross_entropy
sys.path

Please compile source files before using functions CUDA extension.


['/usr/lib/python36.zip',
 '/usr/lib/python3.6',
 '/usr/lib/python3.6/lib-dynload',
 '',
 '/usr/local/lib/python3.6/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/pointnet2-0.0.0-py3.6-linux-x86_64.egg',
 '/usr/local/lib/python3.6/dist-packages/knn_pytorch-0.1-py3.6-linux-x86_64.egg',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.6/dist-packages/IPython/extensions',
 '/home/po/.ipython',
 '/home/po/TM5/s4g-release/inference/grasp_proposal/network_models',
 '/home/po/TM5/graspnetAPI/graspnetAPI',
 '/home/po/TM5']

In [2]:
sys.path.append('/home/po/TM5/graspnet-baseline')
import scipy.io as scio
from dataset.graspnet_dataset1 import GraspNetDataset, collate_fn, load_grasp_labels
from PIL import Image

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


WARNING - 2021-11-29 22:53:29,218 - rigid_transformations - Failed to import geometry msgs in rigid_transformations.py.
WARNING - 2021-11-29 22:53:29,219 - rigid_transformations - Failed to import ros dependencies in rigid_transforms.py
WARNING - 2021-11-29 22:53:29,219 - rigid_transformations - autolab_core not installed as catkin package, RigidTransform ros methods will be unavailable


In [ ]:
class PointNet2(nn.Module):
    """PointNet++ part segmentation with single-scale grouping

    PointNetSA: PointNet Set Abstraction Layer
    PointNetFP: PointNet Feature Propagation Layer

    Args:
        score_classes (int): the number of grasp score classes
        num_centroids (tuple of int): the numbers of centroids to sample in each set abstraction module
        radius (tuple of float): a tuple of radius to query neighbours in each set abstraction module
        num_neighbours (tuple of int): the numbers of neighbours to query for each centroid
        sa_channels (tuple of tuple of int): the numbers of channels within each set abstraction module
        fp_channels (tuple of tuple of int): the numbers of channels for feature propagation (FP) module
        num_fp_neighbours (tuple of int): the numbers of nearest neighbor used in FP
        seg_channels (tuple of int): the numbers of channels in segmentation mlp
        dropout_prob (float): the probability to dropout input features

    References:
        https://github.com/charlesq34/pointnet2/blob/master/models/pointnet2_part_seg.py

    """
    _SA_MODULE = PointNetSAModule
    _FP_MODULE = PointnetFPModule

    def __init__(self,
                 score_classes,
                 num_centroids=(10240, 1024, 128, 0),
                 radius=(0.2, 0.3, 0.4, -1.0),
                 num_neighbours=(64, 64, 64, -1),
                 sa_channels=((32, 32, 64), (64, 64, 128), (128, 128, 256), (256, 512, 1024)),
                 fp_channels=((256, 256), (256, 128), (128, 128), (64, 64, 64)),
                 num_fp_neighbours=(0, 3, 3, 3),
                 seg_channels=(128,),
                 num_removal_directions=5,
                 dropout_prob=0.5):
        super(PointNet2, self).__init__()

        # Sanity check
        num_sa_layers = len(num_centroids)
        num_fp_layers = len(fp_channels)
        assert len(radius) == num_sa_layers
        assert len(num_neighbours) == num_sa_layers
        assert len(sa_channels) == num_sa_layers
        assert num_sa_layers == num_fp_layers
        assert len(num_fp_neighbours) == num_fp_layers

        # Set Abstraction Layers
        feature_channels = 0
        self.sa_modules = nn.ModuleList()
        for ind in range(num_sa_layers):
            sa_module = self._SA_MODULE(in_channels=feature_channels,
                                        mlp_channels=sa_channels[ind],
                                        num_centroids=num_centroids[ind],
                                        radius=radius[ind],
                                        num_neighbours=num_neighbours[ind],
                                        use_xyz=True)
            self.sa_modules.append(sa_module)
            feature_channels = sa_channels[ind][-1]

        inter_channels = [0]
        inter_channels.extend([x[-1] for x in sa_channels])

        # Feature Propagation Layers
        self.fp_modules = nn.ModuleList()
        feature_channels = inter_channels[-1]
        for ind in range(num_fp_layers):
            fp_module = self._FP_MODULE(in_channels=feature_channels + inter_channels[-2 - ind],
                                        mlp_channels=fp_channels[ind],
                                        num_neighbors=num_fp_neighbours[ind])
            self.fp_modules.append(fp_module)
            feature_channels = fp_channels[ind][-1]

        # MLP
        self.mlp_seg = SharedMLP(feature_channels, seg_channels, ndim=1, dropout_prob=dropout_prob)
        self.seg_logit = nn.Conv1d(seg_channels[-1], score_classes, 1, bias=True)

#         self.mlp_grasp_eval = SharedMLP(feature_channels + 28, seg_channels, ndim=2, dropout_prob=dropout_prob)
#         self.grasp_eval_logit = nn.Conv2d(seg_channels[-1], 1, 1, bias=True)
    
        self.mlp_R = SharedMLP(feature_channels, seg_channels, ndim=1)
        self.R_logit = nn.Conv1d(seg_channels[-1], 4, 1, bias=True)

        self.mlp_t = SharedMLP(feature_channels, seg_channels, ndim=1)
        self.t_logit = nn.Conv1d(seg_channels[-1], 3, 1, bias=True)

        self.mlp_movable = SharedMLP(feature_channels, seg_channels, ndim=1, dropout_prob=dropout_prob)
        self.movable_logit = nn.Sequential(
            nn.Conv1d(seg_channels[-1], num_removal_directions, 1, bias=True),
            nn.Sigmoid())

        self.init_weights()
    def forward(self, data_batch):
        points = data_batch['scene_points']

        xyz = points
        feature = None

        # save intermediate results
        inter_xyz = [xyz]
        inter_feature = [feature]

        # Set Abstraction Layers
        for sa_module in self.sa_modules:
            xyz, feature = sa_module(xyz, feature)
            inter_xyz.append(xyz)
            inter_feature.append(feature)

        # Feature Propagation Layers
        sparse_xyz = xyz
        sparse_feature = feature
        for fp_ind, fp_module in enumerate(self.fp_modules):
            dense_xyz = inter_xyz[-2 - fp_ind]
            dense_feature = inter_feature[-2 - fp_ind]
            fp_feature = fp_module(dense_xyz, sparse_xyz, dense_feature, sparse_feature)
            sparse_xyz = dense_xyz
            sparse_feature = fp_feature

        # MLP
        x = self.mlp_seg(sparse_feature)
        logits = self.seg_logit(x)
        
        R = self.mlp_R(sparse_feature)
        R = self.R_logit(R)
        # R = toRotMatrix(R)
        # R = euler2RotMatrix(R)

        t = self.mlp_t(sparse_feature)
        t = self.t_logit(t)

        # t = points + t

        mov = self.mlp_movable(sparse_feature)
        mov = self.movable_logit(mov)  # (B, 5, N)
        
#         local_search_frame = torch.cat([R, t], dim=1).unsqueeze(-1)
#         local_search_frame = local_search_frame.repeat(1, 4, 1, 1)
#         sparse_feature = sparse_feature.unsqueeze(-1)
#         valid_feature = torch.cat([sparse_feature, local_search_frame], dim=1)
#         local_search_logit = self.grasp_eval_logit(self.mlp_grasp_eval(valid_feature))
            
        preds = {"score": logits,
                 "frame_R": R,
                 "frame_t": t,
                 "movable_logits": mov,
                 }

        return preds

    def init_weights(self):
        # nn_utils.init.zeros_(self.t_logit.weight)
        # nn_utils.init.zeros_(self.t_logit.bias)
        pass

In [ ]:
class PointNet2Loss(nn.Module):
    def __init__(self, label_smoothing=0, neg_weight=0.1):
        super(PointNet2Loss, self).__init__()
        self.label_smoothing = label_smoothing
        self.neg_weight = neg_weight

    def forward(self, preds, labels):
        scene_score_logits = preds["scene_score_logits"]  # (B, C, N2)
        score_classes = scene_score_logits.shape[1]
        weight = torch.ones(score_classes, device=scene_score_logits.device)
        weight[0] = self.neg_weight

        movable_logits = preds["movable_logits"]
        movable_labels = labels["scene_movable_labels"]
        mov_loss = F.l1_loss(movable_logits, movable_labels)
        # mov_weight = torch.ones(2, device=movable_logits.device)
        # mov_weight[0] = 0.9

        scene_score_labels = labels["scene_score_labels"]  # (B, N)

        if self.label_smoothing > 0:
            selected_logits = scene_score_logits.transpose(1, 2).contiguous().view(-1, score_classes)
            scene_score_labels = scene_score_labels.view(-1)
            cls_loss = smooth_cross_entropy(selected_logits, scene_score_labels, self.label_smoothing,
                                            weight=weight)

            # movable_logits = movable_logits.transpose(1, 2).contiguous().view(-1, 2)
            # movable_labels = movable_labels.view(-1)
            #
            # mov_loss = smooth_cross_entropy(movable_logits, movable_labels, self.label_smoothing, weight=mov_weight)
        else:
            cls_loss = F.cross_entropy(scene_score_logits, scene_score_labels, weight)

        gt_frame_R = labels["best_frame_R"]
        num_frame_points = gt_frame_R.shape[2]
        pred_frame_R = preds["frame_R"][:, :, :num_frame_points]
        R_loss_1 = ((pred_frame_R - gt_frame_R) ** 2).mean(1, True)
        gt_frame_R_inv = gt_frame_R.clone()
        gt_frame_R_inv[:, 1:3, :] = - gt_frame_R_inv[:, 1:3, :]
        gt_frame_R_inv[:, 4:6, :] = - gt_frame_R_inv[:, 4:6, :]
        gt_frame_R_inv[:, 7:9, :] = - gt_frame_R_inv[:, 7:9, :]
        R_loss_2 = ((pred_frame_R - gt_frame_R_inv) ** 2).mean(1, True)
        R_loss, _ = torch.min(torch.cat([R_loss_1, R_loss_2], dim=1), dim=1)

        # weight loss according to gt_score
        gt_scene_score = labels["scene_score"][:, :num_frame_points]
        R_loss = (R_loss * gt_scene_score).mean() * 5.0
        # gt_norm = torch.stack([gt_frame_R[:, 0, :], gt_frame_R[:, 3, :], gt_frame_R[:, 6, :]], dim=1)
        # pred_norm = torch.stack([pred_frame_R[:, 0, :], pred_frame_R[:, 3, :], pred_frame_R[:, 6, :]], dim=1)
        # norm_loss = torch.mean((pred_norm - gt_norm) ** 2)

        gt_frame_t = labels["best_frame_t"]
        pred_frame_t = preds["frame_t"][:, :, :num_frame_points]
        # t_loss = torch.mean(((pred_frame_t - gt_frame_t) ** 2).sum(1) * gt_scene_score) * 20.0
        t_loss = F.cross_entropy(pred_frame_t, gt_frame_t) * 0.2

        loss_dict = {"cls_loss": cls_loss,
                     "R_loss": R_loss,
                     # "norm_loss": norm_loss,
                     "t_loss": t_loss,
                     "mov_loss": mov_loss,
                     }

        return loss_dict


In [ ]:
class PointNet2Metric(nn.Module):
    def forward(self, preds, labels):
        scene_score_logits = preds["scene_score_logits"]  # (B, C, N2)
        score_classes = scene_score_logits.shape[1]

        scene_score_labels = labels["scene_score_labels"]  # (B, N)

        selected_preds = scene_score_logits.argmax(1).view(-1)
        scene_score_labels = scene_score_labels.view(-1)

        cls_acc = selected_preds.eq(scene_score_labels).float()

        movable_logits = preds["movable_logits"]
        movable_labels = labels["scene_movable_labels"]
        movable_preds = (movable_logits > 0.5).view(-1).int()
        movable_labels = movable_labels.view(-1).int()
        mov_acc = movable_preds.eq(movable_labels).float()

        gt_frame_R = labels["best_frame_R"]
        batch_size, _, num_frame_points = gt_frame_R.shape
        pred_frame_R = preds["frame_R"][:, :, :num_frame_points]
        gt_frame_R = gt_frame_R.transpose(1, 2).contiguous().view(batch_size * num_frame_points, 3, 3)
        gt_frame_R_inv = gt_frame_R.clone()
        gt_frame_R_inv[:, :, 1:] = -gt_frame_R_inv[:, :, 1:]
        pred_frame_R = pred_frame_R.transpose(1, 2).contiguous().view(batch_size * num_frame_points, 3, 3)
        M = torch.bmm(gt_frame_R, pred_frame_R.transpose(1, 2))
        angle = torch.acos(torch.clamp((M[:, 0, 0] + M[:, 1, 1] + M[:, 2, 2] - 1.0) / 2.0, -1.0, 1.0))
        M_inv = torch.bmm(gt_frame_R_inv, pred_frame_R.transpose(1, 2))
        angle_inv = torch.acos(torch.clamp((M_inv[:, 0, 0] + M_inv[:, 1, 1] + M_inv[:, 2, 2] - 1.0) / 2.0, -1.0, 1.0))

        angle_min = torch.stack([angle, angle_inv], dim=1).min(1)[0]
        gt_scene_score = labels["scene_score"][:, :num_frame_points].contiguous().view(-1)
        angle_min = (gt_scene_score * angle_min).mean()

        gt_frame_t = labels["best_frame_t"].view(-1)
        pred_frame_t = preds["frame_t"][:, :, :num_frame_points]
        pred_frame_t = torch.argmax(pred_frame_t, dim=1).view(-1)
        t_acc = pred_frame_t.eq(gt_frame_t).float()

        # t_err = torch.mean(torch.sqrt(((gt_frame_t - pred_frame_t) ** 2).sum(1)))

        return {"cls_acc": cls_acc,
                "mov_acc": mov_acc,
                "R_err": angle_min,
                "t_acc": t_acc,
                }


In [ ]:
# def build_pointnet2_cls(cfg):
def build_model(cfg):
    net = PointNet2(
        score_classes=cfg.DATA.SCORE_CLASSES,
        num_centroids=cfg.MODEL.PN2.NUM_CENTROIDS,
        radius=cfg.MODEL.PN2.RADIUS,
        num_neighbours=cfg.MODEL.PN2.NUM_NEIGHBOURS,
        sa_channels=cfg.MODEL.PN2.SA_CHANNELS,
        fp_channels=cfg.MODEL.PN2.FP_CHANNELS,
        num_fp_neighbours=cfg.MODEL.PN2.NUM_FP_NEIGHBOURS,
        seg_channels=cfg.MODEL.PN2.SEG_CHANNELS,
        num_removal_directions=cfg.DATA.NUM_REMOVAL_DIRECTIONS,
        dropout_prob=cfg.MODEL.PN2.DROPOUT_PROB,
    )

    loss_func = PointNet2Loss(
        label_smoothing=cfg.MODEL.PN2.LABEL_SMOOTHING,
        neg_weight=cfg.MODEL.PN2.NEG_WEIGHT,
    )
    metric = PointNet2Metric()

    return net, loss_func, metric


In [ ]:
root = '/home/po/TM5/graspnetAPI'
valid_obj_idxs, grasp_labels = load_grasp_labels(root)
train_dataset = GraspNetDataset(root, valid_obj_idxs, grasp_labels, split='train', remove_outlier=True, remove_invisible=True, num_points=20000)
print(len(train_dataset))

In [ ]:
# grasp_list = train_dataset.__getitem__(0)['grasp_list']
# point_clouds = train_dataset.__getitem__(0)['point_clouds']

In [ ]:
# print(grasp_list.shape,point_clouds.shape)

In [3]:
# train.py
import os
import sys
import numpy as np
from datetime import datetime
import argparse

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import DataLoader
# from torch.utils.tensorboard import SummaryWriter
# Init datasets and dataloaders 
def my_worker_init_fn(worker_id):
    np.random.seed(np.random.get_state()[1][0] + worker_id)
    pass

In [ ]:
TRAIN_DATALOADER = DataLoader(train_dataset, batch_size=2, shuffle=True,
    num_workers=4, worker_init_fn=my_worker_init_fn, collate_fn=collate_fn)

In [4]:
# proposal_test.py
import sys,os
sys.path.append('/home/po/TM5/s4g-release/inference')
import numpy as np
import open3d
import time
import torch
import torch.nn as nn
from grasp_proposal.cloud_processor.cloud_processor import CloudPreProcessor
from grasp_proposal.configs.yacs_config import load_cfg_from_file
# from grasp_proposal.network_models.models.build_model import build_model
from grasp_proposal.utils.checkpoint import CheckPointer
from grasp_proposal.utils.file_logger_cls import loggin_to_file
from grasp_proposal.utils.grasp_visualizer import GraspVisualizer
from grasp_proposal.utils.logger import setup_logger, MetricLogger

In [5]:
# proposal_test.py
# load_static batch data
# 
cfg_path = "/home/po/TM5/s4g-release/inference/grasp_proposal/configs/curvature_model.yaml"
cfg = load_cfg_from_file(cfg_path)
cfg.defrost()
# cfg.TEST.WEIGHT = cfg.TEST.WEIGHT.replace("${PROJECT_HOME}", os.path.join(os.getcwd(), "../"))
# cfg.TEST.WEIGHT = '/home/po/TM5/s4g-release/inference/trained_models/curvature_model.pth'
cfg.TEST.WEIGHT = cfg.TEST.WEIGHT.replace("${PROJECT_HOME}", os.path.join('/home/po/TM5/s4g-release/inference'))
cfg.freeze()
assert cfg.TEST.BATCH_SIZE == 1

output_dir = "./output"
os.makedirs(output_dir, exist_ok=True)

logger = setup_logger("S4G", output_dir, "unit_test")
logger.info("Using {} of GPUs".format(torch.cuda.device_count()))
logger.info("Load config file from {}".format(cfg_path))
logger.debug("Running with config \n {}".format(cfg))

model, _, _ = build_model(cfg)


2021-11-29 22:53:44,001 S4G INFO: Using 2 of GPUs
2021-11-29 22:53:44,001 S4G INFO: Load config file from /home/po/TM5/s4g-release/inference/grasp_proposal/configs/curvature_model.yaml


NameError: name 'build_model' is not defined

In [ ]:
model

In [ ]:
if torch.cuda.device_count() > 10:
    model = nn.DataParallel(model).cuda()
elif torch.cuda.device_count() == 2:
    model = model.cuda()

trained_model_path = output_dir
check_pointer = CheckPointer(model, save_dir=trained_model_path, logger=logger)
# if cfg.TEST.WEIGHT:
#     weight_path = cfg.TEST.WEIGHT.replace("@", output_dir)
#     check_pointer.load(weight_path, resume=False)
# else:
#     check_pointer.load(None, resume=True)
    
batch_size = 2
num_points = 20000
score_classes = 3
tmp_dic = next(iter(TRAIN_DATALOADER))
point_clouds_tmp = tmp_dic['point_clouds']
point_clouds = point_clouds_tmp.permute(0,2,1)

grasp_ALL = tmp_dic['grasp_list']

data_batch = {
    "scene_points": point_clouds.float().cuda(),
#     "scene_score_labels": torch.tensor(scene_score_labels).long().cuda(),
#     "frame_index": torch.tensor(frame_index).long().cuda()
}
model.eval()
meters = MetricLogger(delimiter="  ")
pcd = 20000
# data_batch, pcd = load_static_data_batch()
tic = time.time()
with torch.no_grad():
    data_batch = {k: v.cuda(non_blocking=True) for k, v in data_batch.items() if isinstance(v, torch.Tensor)}
    tac = time.time()
    data_time = tac - tic
    predictions = model(data_batch)
#     tic = time.time()
#     batch_time = tic - tac
#     with open("inference_time_{}.txt".format("ours"), "a+") as f:
#         f.write("{:.4f}\n".format(batch_time * 1000.0))
#     meters.update(time=batch_time, data=data_time)

#     logger.info(meters.delimiter.join(["{meters}", ]).format(meters=str(meters), ))

#     top_poses, score = loggin_to_file(data_batch, predictions, 0, output_dir, prefix="test", with_label=False)
#     visualizer = GraspVisualizer(pcd)
#     visualizer.add_multiple_poses(top_poses)
#     visualizer.visualize()

In [ ]:
print(predictions['score'].shape,predictions['frame_R'].shape,predictions['frame_t'].shape,predictions['movable_logits'].shape)

In [8]:
batch_size = 2
num_points = 10000
score_classes = 3
scene_points = np.random.randn(batch_size, 3, num_points)
scene_points.shape

(2, 3, 10000)

In [6]:
batch_size = 2
num_points = 10000
score_classes = 3

import numpy as np
tmp_dic = next(iter(TRAIN_DATALOADER))
point_clouds = tmp_dic['point_clouds']
frame_index = np.stack(
    [np.random.choice(np.arange(num_points), num_frame, replace=False) for _ in range(batch_size)])

scene_score_labels = np.random.randint(0, score_classes - 1, (batch_size, num_frame))
point_clouds = point_clouds.permute(0,2,1)
grasp_ALL = tmp_dic['grasp_list']

data_batch = {
    "scene_points": torch.tensor(scene_points).float().cuda(),
    "scene_score_labels": torch.tensor(scene_score_labels).long().cuda(),
    "frame_index": torch.tensor(frame_index).long().cuda()
}

pn2 = PointNet2(score_classes=score_classes).cuda()
pn2
print(scene_score_labels.shape)
# pn2_loss_fn = PointNet2Loss().cuda()
# pn2_metric = PointNet2Metric().cuda()

# preds = pn2(data_batch)
# print("PointNet2: ")
# for k, v in preds.items():
#     print(k, v.shape)

# loss = pn2_loss_fn(preds, data_batch)
# print(loss)
# metric = pn2_metric(preds, data_batch)
# print(metric)

# sum(loss.values()).backward()

NameError: name 'TRAIN_DATALOADER' is not defined

In [ ]:
sys.path

In [ ]:
sys.path.append('/home/po/TM5/s4g-release/inference')
import numpy as np
import open3d
import time
import torch
import torch.nn as nn
from grasp_proposal.cloud_processor.cloud_processor import CloudPreProcessor
from grasp_proposal.configs.yacs_config import load_cfg_from_file
from grasp_proposal.network_models.models.build_model import build_model
from grasp_proposal.utils.checkpoint import CheckPointer
from grasp_proposal.utils.file_logger_cls import loggin_to_file
from grasp_proposal.utils.grasp_visualizer import GraspVisualizer
from grasp_proposal.utils.logger import setup_logger, MetricLogger

In [ ]:
# single_training_data = np.load("/home/po/TM5/s4g-release/2638_view_0.p", allow_pickle=True)
# cloud_array = single_training_data["point_cloud"]
# cloud = CloudPreProcessor(open3d.geometry.PointCloud(open3d.utility.Vector3dVector(cloud_array.T)), False)
# # do not filter workspace here since training data
# cloud.voxelize()
# cloud.remove_outliers()
# points = np.asarray(cloud.pcd.points)
# if points.shape[0] > 25600:
#     random_index = np.random.choice(np.arange(points.shape[0]), 25600, replace=False)
# else:
#     random_index = np.random.choice(np.arange(points.shape[0]), 25600, replace=True)

# points = points[random_index, :]
# C = torch.tensor(points, dtype=torch.float32).unsqueeze(0).transpose(1, 2)
# B = torch.tensor(points, dtype=torch.float32).unsqueeze(0)
# A = torch.tensor(points, dtype=torch.float32)
# # data_batch = {"scene_points": torch.tensor(points, dtype=torch.float32).unsqueeze(0).transpose(1, 2)}
# # print(data_batch,cloud.pcd)
# print(A.shape,B.shape,C.shape)

In [ ]:
# single_training_data['point_cloud'].shape

In [ ]:
R = torch.ones(1, 9, 25600) 
t = torch.ones(1, 3, 25600)
local_search_frame = torch.cat([R, t], dim=1).unsqueeze(-1)
local_search_frame = local_search_frame.repeat(1, 4, 1, 1)
# sparse_feature = sparse_feature.unsqueeze(-1)
# valid_feature = torch.cat([sparse_feature, local_search_frame], dim=1)
# local_search_logit = self.grasp_eval_logit(self.mlp_grasp_eval(valid_feature))

In [ ]:
local_search_frame.shape

In [ ]:
local_search_frame1.shape

In [ ]:

print(torch.__version__)